In [10]:
from pathlib import Path

RAW = Path("../data/raw")  # adjust if your notebook lives elsewhere
p = RAW / "International_airline_activity_Table1_2009toCurrent_0525_Data - Sheet1.csv"

print("CWD:", Path.cwd())
print("RAW exists:", RAW.exists())
print("File exists:", p.exists())
if p.exists():
    print("File size (bytes):", p.stat().st_size)

print("\nRAW folder CSVs:")
for f in RAW.glob("*.csv"):
    print(" -", f.name)

CWD: /Users/kevantamom/Desktop/GITHUB/aviation-insights/notebooks
RAW exists: True
File exists: True
File size (bytes): 1432318

RAW folder CSVs:
 - International_airline_activity_0525_Table4.csv
 - International_airline_activity_0525_Table5.csv
 - International_airline_activity_0525_Table2.csv
 - International_airline_activity_0525_Table3.csv
 - International_airline_activity_0525_Table1.csv
 - International_airline_activity_Table1_2009toCurrent_0525_Data - Sheet1.csv


# t1Hist

In [12]:
import pandas as pd
import numpy as np
import re
from datetime import datetime

def _month_from_text(m):
    # Accept "Jan-09", "January 2009", "Jan", 1..12
    s = str(m).strip()
    try:
        v = int(s)
        return max(1, min(12, v))
    except:
        pass
    try:
        return datetime.strptime(s[:3], "%b").month
    except:
        return np.nan

def _to_number(x: pd.Series) -> pd.Series:
    # Handle "..", blanks, thousands commas
    return (
        x.astype(str)
         .str.strip()
         .replace({"..": np.nan, "—": np.nan, "-": np.nan, "": np.nan})
         .str.replace(",", "", regex=False)
         .astype(float)
    )

def clean_table1_hist(df_raw: pd.DataFrame) -> pd.DataFrame:
    # Standardise headers (keep original labels, just trim)
    df = df_raw.rename(columns=lambda c: str(c).strip())

    # Map your exact column names (with regex fallbacks if labels vary)
    col_map = {
        "Month":           [c for c in df.columns if re.fullmatch(r"(?i)month", c)],
        "Airline_Name":    [c for c in df.columns if re.fullmatch(r"(?i)(scheduled\s+operator|airline|carrier)", c)],
        "Country":         [c for c in df.columns if re.fullmatch(r"(?i)country\s*to/?from", c)],
        "Passengers In":   [c for c in df.columns if re.fullmatch(r"(?i)passengers?\s*in", c)],
        "Freight In":      [c for c in df.columns if re.fullmatch(r"(?i)freight\s*in", c)],
        "Mail In":         [c for c in df.columns if re.fullmatch(r"(?i)mail\s*in", c)],
        "Passengers Out":  [c for c in df.columns if re.fullmatch(r"(?i)passengers?\s*out", c)],
        "Freight Out":     [c for c in df.columns if re.fullmatch(r"(?i)freight\s*out", c)],
        "Mail Out":        [c for c in df.columns if re.fullmatch(r"(?i)mail\s*out", c)],
        "Year":            [c for c in df.columns if re.fullmatch(r"(?i)year", c)],
    }
    pick = lambda k: col_map[k][0] if col_map[k] else None

    m_col   = pick("Month")
    a_col   = pick("Airline_Name")
    c_col   = pick("Country")
    y_col   = pick("Year")
    pin_col = pick("Passengers In")
    fin_col = pick("Freight In")
    min_col = pick("Mail In")
    pout_col= pick("Passengers Out")
    fout_col= pick("Freight Out")
    mout_col= pick("Mail Out")

    # Keep only available columns
    keep = [x for x in [m_col,a_col,c_col,y_col,pin_col,fin_col,min_col,pout_col,fout_col,mout_col] if x]
    df = df[keep].copy()

    # Rename to canonical
    ren = {}
    if a_col:   ren[a_col] = "Airline_Name"
    if c_col:   ren[c_col] = "Country"
    if y_col:   ren[y_col] = "Year"
    if m_col:   ren[m_col] = "MonthText"
    if pin_col: ren[pin_col] = "Passengers_In"
    if fin_col: ren[fin_col] = "Freight_In"
    if min_col: ren[min_col] = "Mail_In"
    if pout_col:ren[pout_col] = "Passengers_Out"
    if fout_col:ren[fout_col] = "Freight_Out"
    if mout_col:ren[mout_col] = "Mail_Out"
    df = df.rename(columns=ren)

    # Parse Month
    if "MonthText" in df.columns:
        df["Month"] = df["MonthText"].apply(_month_from_text).astype("Int64")
    else:
        df["Month"] = pd.NA

    # Numeric coercion
    for col in ["Passengers_In","Freight_In","Mail_In","Passengers_Out","Freight_Out","Mail_Out"]:
        if col in df.columns:
            df[col] = _to_number(df[col])

    # Build long/tidy output
    keys = [k for k in ["Airline_Name","Country","Year","Month"] if k in df.columns]

    inbound = df[keys].copy()
    if "Passengers_In" in df: inbound["Passengers"] = df["Passengers_In"]
    if "Freight_In"   in df: inbound["Freight_tonnes"] = df["Freight_In"]
    if "Mail_In"      in df: inbound["Mail_tonnes"]    = df["Mail_In"]
    inbound["Direction"] = "Inbound"

    outbound = df[keys].copy()
    if "Passengers_Out" in df: outbound["Passengers"] = df["Passengers_Out"]
    if "Freight_Out"   in df: outbound["Freight_tonnes"] = df["Freight_Out"]
    if "Mail_Out"      in df: outbound["Mail_tonnes"]    = df["Mail_Out"]
    outbound["Direction"] = "Outbound"

    out = pd.concat([inbound, outbound], ignore_index=True)

    # Tag & clean "ALL SERVICES" (if ever present)
    out["Is_Total_AllServices"] = out.get("Airline_Name", pd.Series("", index=out.index)).astype(str)\
        .str.contains("ALL SERVICES", case=False, na=False)
    out["Airline_Name"] = out.get("Airline_Name", pd.Series("", index=out.index)).astype(str)\
        .str.replace(r"\s*ALL SERVICES\s*", "", regex=True).str.strip()

    # Build YearMonthDate
    if "Year" in out.columns:
        out["Month"] = out["Month"].fillna(1).astype("Int64")
        out["YearMonthDate"] = pd.to_datetime(
            dict(year=out["Year"], month=out["Month"].astype("int"), day=1), errors="coerce"
        )
    else:
        out["YearMonthDate"] = pd.NaT

    # Final ordering
    cols = ["Year","Month","YearMonthDate","Airline_Name","Country","Direction",
            "Passengers","Freight_tonnes","Mail_tonnes","Is_Total_AllServices"]
    out = out[[c for c in cols if c in out.columns]].sort_values(
        [c for c in ["Year","Month","Airline_Name","Country","Direction"] if c in out.columns]
    ).reset_index(drop=True)

    return out

In [13]:
import pandas as pd

hist_raw = pd.read_csv("../data/raw/International_airline_activity_Table1_2009toCurrent_0525_Data - Sheet1.csv")
t1_hist  = clean_table1_hist(hist_raw)

# table1

In [14]:
import pandas as pd
import numpy as np
from datetime import datetime

def _to_number(s: pd.Series) -> pd.Series:
    return (
        s.astype(str)
         .str.strip()
         .replace({"..": np.nan, "—": np.nan, "-": np.nan, "": np.nan})
         .str.replace(",", "", regex=False)
         .astype(float)
    )

def clean_table1_may_from_sample(df_raw: pd.DataFrame, year=2025, month=5) -> pd.DataFrame:
    # 1) Standardise headers (trim + underscores)
    df = df_raw.rename(columns=lambda c: str(c).strip().replace(" ", "_")).copy()

    # 2) Rename key columns to canonical names
    ren = {
        "Scheduled_Operator": "Airline_Name",
        "Country_to/from": "Country",
        "Inbound_Passengers": "Passengers_In",
        "Inbound_Freight": "Freight_In",
        "Inbound_Mail": "Mail_In",
        "Outbound_Passengers": "Passengers_Out",
        "Outbound_Freight": "Freight_Out",
        "Outbound_Mail": "Mail_Out",
    }
    df = df.rename(columns={k: v for k, v in ren.items() if k in df.columns})

    # 3) Coerce numerics
    for c in ["Passengers_In","Freight_In","Mail_In","Passengers_Out","Freight_Out","Mail_Out"]:
        if c in df.columns:
            df[c] = _to_number(df[c])

    # 4) Inject Year/Month + clean airline / totals flag
    df["Year"] = year
    df["Month"] = month
    df["Is_Total_AllServices"] = (
        df.get("Airline_Name", pd.Series("", index=df.index)).astype(str).str.contains("ALL SERVICES", case=False, na=False) |
        df.get("Country", pd.Series("", index=df.index)).astype(str).str.contains("ALL SERVICES", case=False, na=False)
    )
    # Strip "ALL SERVICES" text from Airline_Name
    if "Airline_Name" in df.columns:
        df["Airline_Name"] = df["Airline_Name"].astype(str).str.replace(r"\s*ALL SERVICES\s*", "", regex=True).str.strip()
    # If Country says ALL SERVICES, null it (to avoid double-counting by country)
    if "Country" in df.columns:
        df.loc[df["Country"].astype(str).str.contains("ALL SERVICES", case=False, na=False), "Country"] = np.nan

    # 5) Build long/tidy (Inbound + Outbound)
    keys = ["Airline_Name","Country","Year","Month"]
    keys = [k for k in keys if k in df.columns]

    inbound = df[keys].copy()
    inbound["Direction"] = "Inbound"
    inbound["Passengers"] = df.get("Passengers_In", np.nan)
    inbound["Freight_tonnes"] = df.get("Freight_In", np.nan)
    inbound["Mail_tonnes"] = df.get("Mail_In", np.nan)

    outbound = df[keys].copy()
    outbound["Direction"] = "Outbound"
    outbound["Passengers"] = df.get("Passengers_Out", np.nan)
    outbound["Freight_tonnes"] = df.get("Freight_Out", np.nan)
    outbound["Mail_tonnes"] = df.get("Mail_Out", np.nan)

    out = pd.concat([inbound, outbound], ignore_index=True)

    # 6) Bring across the totals flag
    out["Is_Total_AllServices"] = df["Is_Total_AllServices"].repeat(2).reset_index(drop=True)

    # 7) YearMonthDate and final order
    out["YearMonthDate"] = pd.to_datetime(dict(year=out["Year"], month=out["Month"], day=1), errors="coerce")
    cols = ["Year","Month","YearMonthDate","Airline_Name","Country","Direction",
            "Passengers","Freight_tonnes","Mail_tonnes","Is_Total_AllServices"]
    out = out[cols].sort_values(["Airline_Name","Country","Direction"], na_position="last").reset_index(drop=True)
    return out

In [15]:
# Load your raw file
raw_path = "../data/raw/International_airline_activity_0525_Table1.csv"
t1_raw = pd.read_csv(raw_path)

# Clean it (set year/month if not May 2025)
t1_may = clean_table1_may_from_sample(t1_raw, year=2025, month=5)

# Save for Power BI
clean_path = "../data/clean/Table1_May2025_clean.csv"
t1_may.to_csv(clean_path, index=False)

print(t1_may.head(8).to_string(index=False))
print("Rows:", len(t1_may))

 Year  Month YearMonthDate                Airline_Name       Country Direction  Passengers  Freight_tonnes  Mail_tonnes  Is_Total_AllServices
 2025      5    2025-05-01 Air Caledonie International New Caledonia   Inbound      2653.0             2.1          0.2                 False
 2025      5    2025-05-01 Air Caledonie International New Caledonia  Outbound      2500.0            40.2          1.9                 False
 2025      5    2025-05-01                  Air Canada        Canada   Inbound     12287.0           277.0          9.1                 False
 2025      5    2025-05-01                  Air Canada        Canada  Outbound     13749.0           458.2         15.9                  True
 2025      5    2025-05-01                   Air China         China   Inbound      7591.0           253.0          NaN                 False
 2025      5    2025-05-01                   Air China         China  Outbound      8098.0           132.4          1.9                  True
 2025 

# Table 2

In [16]:
def _to_percent_norm(s: pd.Series) -> pd.Series:
    """Accepts values like 45.2 (pp), 0.9, '12.3%', returns 0..1 floats."""
    x = s.astype(str).str.strip().str.replace(",", ".", regex=False)
    pct = x.str.endswith("%")
    x = x.str.rstrip("%")
    x = pd.to_numeric(x, errors="coerce")
    # If any value > 1.5 (likely percent points), treat whole column as pp and divide by 100
    if (pd.Series(x).dropna() > 1.5).any() or pct.any():
        x = x / 100.0
    return x

def clean_table2_may_from_sample(df_raw: pd.DataFrame) -> pd.DataFrame:
    # 1) normalise headers
    df = df_raw.rename(columns=lambda c: str(c).strip().replace(" ", "_")).copy()

    # 2) rename key cols
    ren = {
        "Scheduled_Operators": "Airline_Name",
        "Country_to/from": "Country",
        "Total_Passengers_2024": "Passengers_May_LY",
        "Total_Passengers_2025": "Passengers_May",
        "Total_Passengers_Perc_TOTAL": "MarketShare_Passengers",
        "Total_Passengers_Perc_Change": "Passengers_YoY",

        "Total_Freight_(tonnes)_2024": "Freight_2024_t",
        "Total_Freight_(tonnes)_2025": "Freight_2025_t",
        "Total_Freight_(tonnes)_Perc_TOTAL": "MarketShare_Freight",
        "Total_Freight_(tonnes)_Perc_Change": "Freight_YoY",

        "Total_Mail_(tonnes)_2024": "Mail_2024_t",
        "Total_Mail_(tonnes)_2025": "Mail_2025_t",
        "Total_Mail_(tonnes)_Perc_TOTAL": "MarketShare_Mail",
        "Total_Mail_(tonnes)_Perc_Change": "Mail_YoY",
    }
    df.rename(columns={k: v for k, v in ren.items() if k in df.columns}, inplace=True)

    # 3) clean airline names (remove footnote suffixes)
    if "Airline_Name" in df:
        df["Airline_Name"] = (
            df["Airline_Name"]
            .astype(str)
            .str.replace(r"\s*\([a-z]\)\s*$", "", regex=True)  # e.g., " (a)"
            .str.strip()
        )

    # 4) detect & flag totals rows
    df["Is_Total_AllServices"] = (
        df.get("Country", pd.Series("", index=df.index)).astype(str)
          .str.contains("ALL SERVICES", case=False, na=False)
        |
        df.get("Airline_Name", pd.Series("", index=df.index)).astype(str)
          .str.contains("ALL SERVICES", case=False, na=False)
    )

    # 5) numbers & percents
    for c in ["Passengers_May_LY","Passengers_May",
              "Freight_2024_t","Freight_2025_t",
              "Mail_2024_t","Mail_2025_t"]:
        if c in df.columns:
            df[c] = _to_number(df[c])

    for c in ["MarketShare_Passengers","Passengers_YoY",
              "MarketShare_Freight","Freight_YoY",
              "MarketShare_Mail","Mail_YoY"]:
        if c in df.columns:
            df[c] = _to_percent_norm(df[c])

    # 6) optional: drop totals rows from the analysis table
    df_clean = df[~df["Is_Total_AllServices"]].copy()

    # 7) select tidy passenger-focused output (keep freight/mail if you want)
    keep_cols = [c for c in [
        "Airline_Name","Country",
        "Passengers_May","Passengers_May_LY","Passengers_YoY","MarketShare_Passengers",
        "Freight_2024_t","Freight_2025_t","Freight_YoY","MarketShare_Freight",
        "Mail_2024_t","Mail_2025_t","Mail_YoY","MarketShare_Mail"
    ] if c in df_clean.columns]

    out = df_clean[keep_cols].sort_values(["Airline_Name","Country"]).reset_index(drop=True)
    return out

In [17]:
raw_path = "../data/raw/International_airline_activity_0525_Table2.csv"
t2_raw = pd.read_csv(raw_path)

t2_may = clean_table2_may_from_sample(t2_raw)
t2_may.to_csv("../data/clean/Table2_May2025_clean.csv", index=False)

print(t2_may.head(8).to_string(index=False))
print("Rows:", len(t2_may))

               Airline_Name          Country  Passengers_May  Passengers_May_LY  Passengers_YoY  MarketShare_Passengers  Freight_2024_t  Freight_2025_t  Freight_YoY  MarketShare_Freight  Mail_2024_t  Mail_2025_t  Mail_YoY  MarketShare_Mail
Air Caledonie International    New Caledonia          5153.0             3548.0           0.452                   0.002            22.7            42.3        0.863                0.000          0.4          2.1     3.964             0.001
                 Air Canada           Canada         26036.0            27444.0          -0.051                   0.008           906.2           735.2       -0.189                0.008         33.2         25.0    -0.247             0.012
                  Air China            China         15689.0            17975.0          -0.127                   0.005           722.4           385.3       -0.467                0.004          2.8          1.9    -0.329             0.001
                  Air India            I

# table 3

In [18]:
def _to_percent_norm(s: pd.Series) -> pd.Series:
    x = s.astype(str).str.strip().str.replace(",", ".", regex=False)
    x = x.str.rstrip("%")
    x = pd.to_numeric(x, errors="coerce")
    # If values look like 0–100, convert to 0–1
    return np.where(x > 1.5, x/100.0, x)

def clean_table3_may_from_sample(df_raw: pd.DataFrame, year=2025, month=5) -> pd.DataFrame:
    # 1) Standardise headers
    df = df_raw.rename(columns=lambda c: str(c).strip().replace(" ", "_")).copy()

    # 2) Rename to canonical names
    ren = {
        "Scheduled_Operator": "Airline_Name",
        "Service_to/from": "Service_Region",

        "Inbound_No._of_Flights": "In_Flights",
        "Inbound_Pax_Carried": "In_Passengers",
        "Inbound_Seats_Available": "In_Seats",
        "Inbound_Seat_Utilisation_%": "In_LoadFactor",

        "Outbound_No._of_Flights": "Out_Flights",
        "Outbound_Pax_Carried": "Out_Passengers",
        "Outbound_Seats_Available": "Out_Seats",
        "Outbound_Seat_Utilisation_%": "Out_LoadFactor",
    }
    df.rename(columns={k: v for k, v in ren.items() if k in df.columns}, inplace=True)

    # 3) Clean airline (remove trailing footnote markers like " (a)")
    if "Airline_Name" in df:
        df["Airline_Name"] = (
            df["Airline_Name"]
            .astype(str)
            .str.replace(r"\s*\([a-z]\)\s*$", "", regex=True)
            .str.replace(r"\s*ALL SERVICES\s*", "", regex=True)
            .str.strip()
        )

    # 4) Flag totals rows
    df["Is_Total_AllServices"] = (
        df.get("Service_Region", pd.Series("", index=df.index)).astype(str)
          .str.contains("ALL SERVICES", case=False, na=False)
        |
        df.get("Airline_Name", pd.Series("", index=df.index)).astype(str)
          .str.contains("ALL SERVICES", case=False, na=False)
    )

    # 5) Numeric coercion
    for c in ["In_Flights","In_Passengers","In_Seats","Out_Flights","Out_Passengers","Out_Seats"]:
        if c in df.columns:
            df[c] = _to_number(df[c])

    for c in ["In_LoadFactor","Out_LoadFactor"]:
        if c in df.columns:
            df[c] = _to_percent_norm(df[c])

    # 6) Add Year/Month
    df["Year"] = year
    df["Month"] = month

    # 7) Build tidy long format: Inbound + Outbound
    keys = [k for k in ["Airline_Name","Service_Region","Year","Month"] if k in df.columns]

    inbound = df[keys].copy()
    inbound["Direction"] = "Inbound"
    inbound["Flights"] = df.get("In_Flights", np.nan)
    inbound["Passengers"] = df.get("In_Passengers", np.nan)
    inbound["Seats_Available"] = df.get("In_Seats", np.nan)
    inbound["Seat_Utilisation"] = df.get("In_LoadFactor", np.nan)

    outbound = df[keys].copy()
    outbound["Direction"] = "Outbound"
    outbound["Flights"] = df.get("Out_Flights", np.nan)
    outbound["Passengers"] = df.get("Out_Passengers", np.nan)
    outbound["Seats_Available"] = df.get("Out_Seats", np.nan)
    outbound["Seat_Utilisation"] = df.get("Out_LoadFactor", np.nan)

    out = pd.concat([inbound, outbound], ignore_index=True)

    # 8) Carry totals flag & YearMonthDate
    out["Is_Total_AllServices"] = df["Is_Total_AllServices"].repeat(2).reset_index(drop=True)
    out["YearMonthDate"] = pd.to_datetime(dict(year=out["Year"], month=out["Month"], day=1), errors="coerce")

    # 9) Final ordering (drop totals rows for analysis if you want)
    out = out[[
        "Year","Month","YearMonthDate",
        "Airline_Name","Service_Region","Direction",
        "Flights","Passengers","Seats_Available","Seat_Utilisation",
        "Is_Total_AllServices"
    ]]

    # If you want to exclude ALL SERVICES totals from visuals:
    out = out[~out["Is_Total_AllServices"]].reset_index(drop=True)

    return out

In [19]:
t3_raw = pd.read_csv("../data/raw/International_airline_activity_0525_Table3.csv")
t3_may = clean_table3_may_from_sample(t3_raw, year=2025, month=5)

t3_may.to_csv("../data/clean/Table3_May2025_clean.csv", index=False)
print(t3_may.head(8).to_string(index=False))
print("Rows:", len(t3_may))

 Year  Month YearMonthDate                Airline_Name   Service_Region Direction  Flights  Passengers  Seats_Available  Seat_Utilisation  Is_Total_AllServices
 2025      5    2025-05-01 Air Caledonie International    New Caledonia   Inbound     17.0      2653.0           3643.0             0.728                 False
 2025      5    2025-05-01                  Air Canada           Canada   Inbound     53.0     12287.0          15941.0             0.771                 False
 2025      5    2025-05-01                   Air China            China   Inbound     31.0      7591.0           9031.0             0.841                 False
 2025      5    2025-05-01                   Air India            India   Inbound     62.0     14028.0          15872.0             0.884                 False
 2025      5    2025-05-01               Air Mauritius        Mauritius   Inbound      8.0      1788.0           2350.0             0.761                 False
 2025      5    2025-05-01             A

# TABLE 4

In [20]:
import pandas as pd
import numpy as np
from pathlib import Path

RAW   = Path("../data/raw")
CLEAN = Path("../data/clean")
CLEAN.mkdir(parents=True, exist_ok=True)

# ---------- helpers ----------
def to_number(s: pd.Series) -> pd.Series:
    """'1,234' -> 1234.0 ; '..'/'-' -> NaN"""
    return (
        s.astype(str)
         .str.strip()
         .replace({"..": np.nan, "—": np.nan, "-": np.nan, "": np.nan})
         .str.replace(",", "", regex=False)
         .astype(float)
    )

def to_percent_0_1(s: pd.Series) -> pd.Series:
    """26.1 or '26.1%' -> 0.261 ; 0.26 stays 0.26"""
    x = s.astype(str).str.strip()
    has_pct = x.str.endswith("%")
    x = x.str.rstrip("%")
    x = pd.to_numeric(x, errors="coerce")
    # If any value looks like percent-points (>1.5), divide by 100
    if pd.notna(x).any() and (x.dropna() > 1.5).any() or has_pct.any():
        x = x / 100.0
    return x

# ---------- cleaner ----------
def clean_table4_from_sample(df_raw: pd.DataFrame) -> pd.DataFrame:
    df = df_raw.copy()
    # standardise headers a bit (keep original labels)
    df.columns = [c.strip() for c in df.columns]

    # replace '..' with NaN across the board
    df = df.replace("..", np.nan)

    # numeric columns (everything except City, Indicator)
    num_cols = [c for c in df.columns if c not in ["City", "Indicator"]]

    # convert numerics
    for c in num_cols:
        df[c] = to_number(df[c])

    # keep % change both as percent-points and 0-1
    for c in ["Inbound_Perc_Change", "Outbound_Perc_Change"]:
        if c in df.columns:
            df[c + "_0_1"] = to_percent_0_1(df[c])

    return df

# ---------- reshape: long ----------
def table4_to_long(df_clean: pd.DataFrame) -> pd.DataFrame:
    """
    Output columns:
    City | Metric | Direction | Year | Value
    Metric ∈ {'Passengers','Freight (Tonnes)','Aircraft Movements'}
    Direction ∈ {'Inbound','Outbound'}
    Year ∈ {2024, 2025}
    """
    # melt inbound/outbound 2024/2025
    long_vals = df_clean.melt(
        id_vars=["City", "Indicator", "Inbound_Perc_Change", "Inbound_Perc_Change_0_1",
                 "Outbound_Perc_Change", "Outbound_Perc_Change_0_1"],
        value_vars=["Inbound_2024","Inbound_2025","Outbound_2024","Outbound_2025"],
        var_name="DirectionYear", value_name="Value"
    )

    # split Direction & Year
    long_vals["Direction"] = np.where(long_vals["DirectionYear"].str.startswith("Inbound"), "Inbound", "Outbound")
    long_vals["Year"] = long_vals["DirectionYear"].str.extract(r"(2024|2025)").astype(int)
    long_vals = long_vals.drop(columns=["DirectionYear"])

    # rename for clarity
    long_vals = long_vals.rename(columns={"Indicator":"Metric"})

    # Optional: bring the corresponding YoY for that direction
    long_vals["Perc_Change_pp"] = np.where(
        long_vals["Direction"]=="Inbound",
        long_vals["Inbound_Perc_Change"],
        long_vals["Outbound_Perc_Change"]
    )
    long_vals["Perc_Change_0_1"] = np.where(
        long_vals["Direction"]=="Inbound",
        long_vals["Inbound_Perc_Change_0_1"],
        long_vals["Outbound_Perc_Change_0_1"]
    )

    # final ordering
    cols = ["City","Metric","Direction","Year","Value","Perc_Change_pp","Perc_Change_0_1"]
    return long_vals[cols].sort_values(["City","Metric","Direction","Year"]).reset_index(drop=True)

# ---------- reshape: wide per city ----------
def table4_to_wide(df_clean: pd.DataFrame) -> pd.DataFrame:
    """
    One row per city. Columns like:
      Passengers_In_2024, Passengers_Out_2024, Freight (Tonnes)_In_2025, ...
      + Perc change columns for inbound/outbound per metric
    """
    # pivot values
    def pivot_metric(metric_name):
        sub = df_clean[df_clean["Indicator"] == metric_name].copy()
        keep = ["City",
                "Inbound_2024","Inbound_2025","Inbound_Perc_Change","Inbound_Perc_Change_0_1",
                "Outbound_2024","Outbound_2025","Outbound_Perc_Change","Outbound_Perc_Change_0_1"]
        sub = sub[keep]
        sub = sub.rename(columns={
            "Inbound_2024": f"{metric_name}_In_2024",
            "Inbound_2025": f"{metric_name}_In_2025",
            "Inbound_Perc_Change": f"{metric_name}_In_YoY_pp",
            "Inbound_Perc_Change_0_1": f"{metric_name}_In_YoY",
            "Outbound_2024": f"{metric_name}_Out_2024",
            "Outbound_2025": f"{metric_name}_Out_2025",
            "Outbound_Perc_Change": f"{metric_name}_Out_YoY_pp",
            "Outbound_Perc_Change_0_1": f"{metric_name}_Out_YoY",
        })
        return sub

    m1 = pivot_metric("Passengers")
    m2 = pivot_metric("Freight (Tonnes)")
    m3 = pivot_metric("Aircraft Movements")

    # merge on City
    wide = m1.merge(m2, on="City", how="outer").merge(m3, on="City", how="outer")
    return wide.sort_values("City").reset_index(drop=True)



In [21]:
# ---------- run ----------
# Replace with your path/file name
t4_raw_path = RAW / "International_airline_activity_0525_Table4.csv"
t4_raw = pd.read_csv(t4_raw_path)

t4_clean = clean_table4_from_sample(t4_raw)
t4_long  = table4_to_long(t4_clean)
t4_wide  = table4_to_wide(t4_clean)

# save
t4_clean.to_csv(CLEAN / "Table4_Cities_clean.csv", index=False)
t4_long.to_csv(CLEAN / "Table4_Cities_long.csv", index=False)
t4_wide.to_csv(CLEAN / "Table4_Cities_wide.csv", index=False)

print("✅ Wrote:",
      CLEAN / "Table4_Cities_clean.csv",
      CLEAN / "Table4_Cities_long.csv",
      CLEAN / "Table4_Cities_wide.csv")

✅ Wrote: ../data/clean/Table4_Cities_clean.csv ../data/clean/Table4_Cities_long.csv ../data/clean/Table4_Cities_wide.csv


# TABLE 5

In [22]:
# ---------- cleaner ----------
def clean_table5_from_sample(df_raw: pd.DataFrame) -> pd.DataFrame:
    df = df_raw.copy()
    # Standardise headers a little
    df.columns = [c.strip().replace(" ", "_") for c in df.columns]

    # Canonical rename
    ren = {
        "Foreign_Port": "Foreign_City",
        "Australian_Port": "Australian_City",
        "Passengers_2024_Inbound":  "Pax_2024_In",
        "Passengers_2024_Outbound": "Pax_2024_Out",
        "Passengers_2025_Inbound":  "Pax_2025_In",
        "Passengers_2025_Outbound": "Pax_2025_Out",
        "Freight_(tonnes)_2024_Inbound":  "Freight_2024_In_t",
        "Freight_(tonnes)_2024_Outbound": "Freight_2024_Out_t",
        "Freight_(tonnes)_2025_Inbound":  "Freight_2025_In_t",
        "Freight_(tonnes)_2025_Outbound": "Freight_2025_Out_t",
    }
    df.rename(columns={k:v for k,v in ren.items() if k in df.columns}, inplace=True)

    # Replace textual missing with NaN
    df = df.replace({"..": np.nan})

    # Numeric coercion
    num_cols = [c for c in df.columns if c not in ["Foreign_City","Australian_City"]]
    for c in num_cols:
        df[c] = to_number(df[c])

    # Flag aggregates (optional: you can filter them out later)
    df["Is_Total_Row"] = df["Foreign_City"].astype(str).str.fullmatch(r"(?i)total") | \
                         df["Australian_City"].astype(str).str.fullmatch(r"(?i)australia")

    # Consistent city text
    for c in ["Foreign_City","Australian_City"]:
        if c in df:
            df[c] = df[c].astype(str).str.strip()

    return df

# ---------- to long (tidy) ----------
def table5_to_long(df_clean: pd.DataFrame) -> pd.DataFrame:
    """
    Output columns:
      Australian_City | Foreign_City | Metric | Direction | Year | Value
    Metric ∈ {'Passengers','Freight_tonnes'}
    Direction ∈ {'Inbound','Outbound'}
    Year ∈ {2024, 2025}
    """
    # Build tidy for passengers
    pax_long = df_clean.melt(
        id_vars=["Australian_City","Foreign_City","Is_Total_Row"],
        value_vars=["Pax_2024_In","Pax_2024_Out","Pax_2025_In","Pax_2025_Out"],
        var_name="key", value_name="Value"
    )
    pax_long["Metric"] = "Passengers"
    pax_long["Direction"] = np.where(pax_long["key"].str.contains("_In$"), "Inbound", "Outbound")
    pax_long["Year"] = pax_long["key"].str.extract(r"(\d{4})").astype(int)
    pax_long = pax_long.drop(columns=["key"])

    # Build tidy for freight (if present)
    freight_cols = [c for c in df_clean.columns if c.startswith("Freight_")]
    if freight_cols:
        fr_long = df_clean.melt(
            id_vars=["Australian_City","Foreign_City","Is_Total_Row"],
            value_vars=freight_cols,
            var_name="key", value_name="Value"
        )
        fr_long["Metric"] = "Freight_tonnes"
        fr_long["Direction"] = np.where(fr_long["key"].str.contains("_In_t$"), "Inbound", "Outbound")
        fr_long["Year"] = fr_long["key"].str.extract(r"(\d{4})").astype(int)
        fr_long = fr_long.drop(columns=["key"])
        long = pd.concat([pax_long, fr_long], ignore_index=True)
    else:
        long = pax_long

    # Sort & return
    long = long[["Australian_City","Foreign_City","Metric","Direction","Year","Value","Is_Total_Row"]]\
             .sort_values(["Australian_City","Foreign_City","Metric","Direction","Year"])\
             .reset_index(drop=True)
    return long

# ---------- quick “top routes 2025” helper ----------
def top_routes_2025_passengers(long_df: pd.DataFrame, top_n=100, exclude_totals=True) -> pd.DataFrame:
    sub = long_df[(long_df["Metric"]=="Passengers") & (long_df["Year"]==2025)].copy()
    if exclude_totals:
        sub = sub[~sub["Is_Total_Row"]]
    # Sum inbound + outbound to get total flow per city pair
    agg = (sub.groupby(["Australian_City","Foreign_City"], as_index=False)
             .agg(Passengers_2025_Total=("Value","sum")))
    return agg.sort_values("Passengers_2025_Total", ascending=False).head(top_n).reset_index(drop=True)



In [23]:
# ---------- run ----------
t5_raw_path = RAW / "International_airline_activity_0525_Table5.csv"  # adjust if needed
t5_raw  = pd.read_csv(t5_raw_path)

t5_clean = clean_table5_from_sample(t5_raw)
t5_long  = table5_to_long(t5_clean)
t5_top   = top_routes_2025_passengers(t5_long, top_n=100, exclude_totals=True)

t5_clean.to_csv(CLEAN / "Table5_pairs_clean.csv", index=False)
t5_long.to_csv(CLEAN / "Table5_pairs_long.csv", index=False)
t5_top.to_csv(CLEAN / "Table5_pairs_toproutes_2025.csv", index=False)

print("✅ Wrote:",
      CLEAN / "Table5_pairs_clean.csv",
      CLEAN / "Table5_pairs_long.csv",
      CLEAN / "Table5_pairs_toproutes_2025.csv")

✅ Wrote: ../data/clean/Table5_pairs_clean.csv ../data/clean/Table5_pairs_long.csv ../data/clean/Table5_pairs_toproutes_2025.csv


1) Build dimensions (once, then reuse)

dim_date (month grain)

In [24]:
import pandas as pd
from datetime import datetime

def build_dim_date(start="2009-01-01", end="2025-12-01"):
    dates = pd.date_range(start=start, end=end, freq="MS")  # Month start
    df = pd.DataFrame({"YearMonthDate": dates})
    df["DateID"]     = df["YearMonthDate"].dt.strftime("%Y%m").astype(int)
    df["Year"]       = df["YearMonthDate"].dt.year
    df["Month"]      = df["YearMonthDate"].dt.month
    df["MonthName"]  = df["YearMonthDate"].dt.strftime("%b")
    df["Quarter"]    = "Q" + ((df["Month"]-1)//3 + 1).astype(str)
    return df

dim_date = build_dim_date(
    start=f"{t1_hist['Year'].min()}-{t1_hist['Month'].min():02d}-01",
    end=f"{t1_hist['Year'].max()}-{t1_hist['Month'].max():02d}-01"
)
dim_date.to_csv("../data/clean/dim_date.csv", index=False)

dim_airline (from Table1 history)

In [32]:
import pandas as pd
import numpy as np
import re
from pathlib import Path

CLEAN = Path("../data/clean")
CLEAN.mkdir(parents=True, exist_ok=True)

# --- 1) helpers ---
def norm_airline(name: str) -> str:
    if pd.isna(name): return None
    s = str(name).strip()
    s = re.sub(r"\s*\([^)]*\)\s*$", "", s)  # drop trailing footnote markers like "(a)"
    s = re.sub(r"\s+", " ", s)
    return s

# Known home-country overrides for major carriers (extend anytime)
HOME_COUNTRY_OVERRIDE = {
    "Qantas Airways": "Australia",
    "Jetstar": "Australia",
    "Virgin Australia": "Australia",
    "Air New Zealand": "New Zealand",
    "Singapore Airlines": "Singapore",
    "Scoot": "Singapore",
    "Silk Air": "Singapore",
    "Cathay Pacific Airways": "Hong Kong (SAR)",
    "Hong Kong Airlines": "Hong Kong (SAR)",
    "Emirates": "United Arab Emirates",
    "Etihad Airways": "United Arab Emirates",
    "Qatar Airways": "Qatar",
    "Turkish Airlines": "Turkey",
    "British Airways": "UK",
    "American Airlines": "USA",
    "Delta Air Lines": "USA",
    "United Airlines": "USA",
    "Air Canada": "Canada",
    "Air China": "China",
    "China Southern Airlines": "China",
    "China Eastern Airlines": "China",
    "Xiamen Airlines": "China",
    "Hainan Airlines": "China",
    "Tianjin Airlines": "China",
    "Juneyao Air": "China",
    "Sichuan Airlines": "China",
    "Japan Airlines": "Japan",
    "All Nippon Airways": "Japan",
    "Korean Air": "Korea",
    "Asiana Airlines": "Korea",
    "Malaysia Airlines": "Malaysia",
    "AirAsia Berhad": "Malaysia",
    "AirAsia X": "Malaysia",
    "Garuda Indonesia": "Indonesia",
    "Batik Air Indonesia": "Indonesia",
    "Indonesia AirAsia": "Indonesia",
    "Philippine Airlines": "Philippines",
    "Cebu Pacific Air": "Philippines",
    "Vietnam Airlines": "Vietnam",
    "Vietjet Air": "Vietnam",
    "SriLankan Airlines": "Sri Lanka",
    "Fiji Airways": "Fiji",
    "Air Vanuatu": "Vanuatu",
    "Nauru Airlines": "Nauru",
    "Solomon Airlines": "Solomon Islands",
    "LATAM Airlines": "Chile",
    "LAN Airlines": "Chile",
    "South African Airways": "South Africa",
    "EVA Air": "Taiwan",
    "China Airlines": "Taiwan",
    "Royal Brunei Airlines": "Brunei",
    "Tasman Cargo Airlines": "Australia",
}

# --- 2) source: use your already-cleaned historical Table 1 ---
# t1_hist must exist in memory. If not, load your cleaned CSV:
# t1_hist = pd.read_csv("../data/clean/Table1_Historical_clean.csv", parse_dates=["YearMonthDate"])

df = t1_hist.copy()

# Normalise names and drop junk
df["Airline_Name"] = df["Airline_Name"].apply(norm_airline)
df = df[~df["Airline_Name"].isin([None, "", "ALL SERVICES"])]

# --- 3) infer home country from most frequent "Country to/from" seen historically ---
# Note: this is a proxy; override map will correct the majors.
mode_country = (
    df.groupby(["Airline_Name","Country"], dropna=False)
      .size()
      .reset_index(name="n")
      .sort_values(["Airline_Name","n"], ascending=[True, False])
      .drop_duplicates(subset=["Airline_Name"])
      .rename(columns={"Country":"Inferred_Home_Country"})
      [["Airline_Name","Inferred_Home_Country"]]
)

# Unique airline list
airlines = (
    df[["Airline_Name"]]
      .drop_duplicates()
      .sort_values("Airline_Name")
      .reset_index(drop=True)
)

# Apply overrides, else use inferred mode
airlines = airlines.merge(mode_country, on="Airline_Name", how="left")
airlines["Home_Country"] = airlines["Airline_Name"].map(HOME_COUNTRY_OVERRIDE)\
                             .fillna(airlines["Inferred_Home_Country"])
airlines.drop(columns=["Inferred_Home_Country"], inplace=True)

# Final tidy: if still missing, label "Unknown" (you can fix manually later)
airlines["Home_Country"] = airlines["Home_Country"].fillna("Unknown")

# Assign surrogate key
airlines.insert(0, "AirlineID", range(1, len(airlines)+1))

dim_airline = airlines[["AirlineID","Airline_Name","Home_Country"]]

# Save
dim_airline.to_csv(CLEAN / "dim_airline.csv", index=False)
print(f"✅ dim_airline rows: {len(dim_airline)}  → {CLEAN/'dim_airline.csv'}")

# Optional: quick sanity check for duplicates
dups = dim_airline["Airline_Name"].duplicated().sum()
print(f"Duplicate Airline_Name rows: {dups}")

✅ dim_airline rows: 97  → ../data/clean/dim_airline.csv
Duplicate Airline_Name rows: 0


2) Build fact tables

fact_airline_monthly (from Table 1 history)

In [46]:
import pandas as pd

t1 = pd.read_csv("../data/clean/Table1_May2025_clean.csv")   # Year, Month, Airline_Name, Country, Direction, Passengers, Freight_tonnes, Mail_tonnes, Is_Total_AllServices
dim_airline = pd.read_csv("../data/clean/dim_airline.csv")
dim_date    = pd.read_csv("../data/clean/dim_date.csv")      # has Year, Month, DateID

# keep only non-total rows
t1 = t1[t1["Is_Total_AllServices"] == False].copy()

# ensure ints for join
for c in ["Year","Month"]:
    t1[c] = pd.to_numeric(t1[c], errors="coerce").astype("Int64")
    dim_date[c] = pd.to_numeric(dim_date[c], errors="coerce").astype("Int64")

f1 = (t1
      .merge(dim_airline[["AirlineID","Airline_Name"]], on="Airline_Name", how="left")
      .merge(dim_date[["DateID","Year","Month"]], on=["Year","Month"], how="left"))

fact_airline_monthly = f1[[
    "DateID","AirlineID","Country","Direction",
    "Passengers","Freight_tonnes","Mail_tonnes"
]]

fact_airline_monthly.to_csv("../data/clean/fact_airline_monthly.csv", index=False)

print(f"✅ fact_airline_monthly rows: {len(fact_airline_monthly)}")
print("Missing AirlineID:", fact_airline_monthly["AirlineID"].isna().sum())
print("Missing DateID:", fact_airline_monthly["DateID"].isna().sum())

✅ fact_airline_monthly rows: 238
Missing AirlineID: 0
Missing DateID: 0


fact_airline_flights (from Table 3 May clean)


In [45]:
import pandas as pd

# --- Load ---
t3 = pd.read_csv("../data/clean/Table3_May2025_clean.csv")  # has Year, Month, Airline_Name, ...
dim_airline = pd.read_csv("../data/clean/dim_airline.csv")
dim_date    = pd.read_csv("../data/clean/dim_date.csv")     # must have Year, Month, DateID

# --- Ensure Year/Month are numeric ---
for c in ["Year", "Month"]:
    if c in t3.columns:
        t3[c] = pd.to_numeric(t3[c], errors="coerce").astype("Int64")
    else:
        # If Month isn't present (e.g., single-month extract), hardcode May=5 or set what applies
        if c == "Year":
            raise ValueError("t3 is missing 'Year' column.")
        if c == "Month":
            t3["Month"] = 5  # set to the correct month for this file

# Same for dim_date
for c in ["Year", "Month"]:
    if c not in dim_date.columns:
        raise ValueError("dim_date must include 'Year' and 'Month' columns.")
    dim_date[c] = pd.to_numeric(dim_date[c], errors="coerce").astype("Int64")

# --- Merge IDs (no YearMonthDate anywhere) ---
f3 = (
    t3
    .merge(dim_airline[["AirlineID","Airline_Name"]], on="Airline_Name", how="left")
    .merge(dim_date[["DateID","Year","Month"]], on=["Year","Month"], how="left")
)

# --- Build fact table ---
fact_airline_flights = f3[[
    "DateID","AirlineID","Service_Region","Direction",
    "Flights","Passengers","Seats_Available","Seat_Utilisation"
]]

fact_airline_flights.to_csv("../data/clean/fact_airline_flights.csv", index=False)

# --- Diagnostics ---
print(f"✅ fact_airline_flights rows: {len(fact_airline_flights)}")
print(f"⚠️ AirlineID missing: {fact_airline_flights['AirlineID'].isna().sum()}")
print(f"⚠️ DateID missing: {fact_airline_flights['DateID'].isna().sum()}")

✅ fact_airline_flights rows: 222
⚠️ AirlineID missing: 0
⚠️ DateID missing: 0


fact_airport_totals (from Table 4 long)

In [48]:
import pandas as pd

# --- Load
t4_long = pd.read_csv("../data/clean/Table4_Cities_long.csv")  # City, Metric, Direction, Year, Value
dim_citycountry = pd.read_csv("../data/clean/dim_citycountry.csv")
dim_date = pd.read_csv("../data/clean/dim_date.csv")

# --- Normalise text
t4_long["City"] = t4_long["City"].astype(str).str.strip().str.replace(r"\s+", " ", regex=True)
dim_citycountry["City"] = dim_citycountry["City"].astype(str).str.strip().str.replace(r"\s+", " ", regex=True)

# --- Map City → CityID
f4 = t4_long.merge(
    dim_citycountry.rename(columns={"City": "CityName"}),
    left_on="City",
    right_on="CityName",
    how="left"
).drop(columns=["CityName"])

# --- Build YearMonthDate and join DateID
f4["YearMonthDate"] = pd.to_datetime(dict(year=f4["Year"], month=1, day=1))
dim_date["YearMonthDate"] = pd.to_datetime(dim_date["YearMonthDate"])
f4 = f4.merge(dim_date[["DateID", "YearMonthDate"]], on="YearMonthDate", how="left")

# --- Final fact table
fact_airport_totals = f4[["DateID", "CityID", "City", "Metric", "Direction", "Value"]] \
    .rename(columns={"Value": "Amount"})

# --- Optional: drop rows with missing CityID
missing_city = fact_airport_totals["CityID"].isna().sum()
if missing_city:
    print(f"⚠️ Missing CityID for {missing_city} rows")
    fact_airport_totals = fact_airport_totals.dropna(subset=["CityID"])

# --- Save
fact_airport_totals.to_csv("../data/clean/fact_airport_totals.csv", index=False)
print(f"✅ fact_airport_totals rows: {len(fact_airport_totals)}")

✅ fact_airport_totals rows: 144


fact_citypairs (from Table 5 long)

In [43]:
import pandas as pd
import re

# --- reload inputs (or reuse your existing variables) ---
t5_long = pd.read_csv("../data/clean/Table5_pairs_long.csv")  # Australian_City, Foreign_City, Metric, Direction, Year, Value
dim_citycountry = pd.read_csv("../data/clean/Dim_CityCountry.csv")  # CityID, City, Country
dim_date = pd.read_csv("../data/clean/dim_date.csv")  # DateID, Year, ...

# --- normalise ---
for c in ["Australian_City","Foreign_City"]:
    t5_long[c] = (t5_long[c].astype(str)
                              .str.strip()
                              .str.replace(r"\s+", " ", regex=True))

dim_citycountry["City"]    = dim_citycountry["City"].astype(str).str.strip().str.replace(r"\s+", " ", regex=True)
dim_citycountry["Country"] = dim_citycountry["Country"].astype(str).str.strip().str.replace(r"\s+", " ", regex=True)

# --- 0) Drop aggregates & subtotal labels ---
# rows where the “city” column is actually country-level or a subtotal
def is_aggregate(label: str) -> bool:
    s = str(label).strip()
    if s.lower() == "australia":        # country total
        return True
    if s.lower().startswith("total"):    # subtotal labels like "Total, Broome"
        return True
    return False

mask_bad = (
    t5_long["Australian_City"].apply(is_aggregate) |
    t5_long["Foreign_City"].apply(is_aggregate)
)
t5_long = t5_long[~mask_bad].copy()

# --- 1) types ---
t5_long["Year"]  = pd.to_numeric(t5_long["Year"], errors="coerce").astype("Int64")
t5_long["Value"] = pd.to_numeric(t5_long["Value"], errors="coerce")

# --- 2) Countries to enable (City,Country) joins ---
t5_long["Australian_Country"] = "Australia"

city_to_country = (dim_citycountry
                   .drop_duplicates(subset=["City"])
                   .set_index("City")["Country"]
                   .to_dict())
t5_long["Foreign_Country"] = t5_long["Foreign_City"].map(city_to_country)

# Optional: quick alias fixes if any spelling mismatches show up later
alias_map = {
    # "Ho Chi Minh": "Ho Chi Minh City",
    # "Xi’an": "Xi'an",
}
for col in ["Australian_City","Foreign_City"]:
    t5_long[col] = t5_long[col].replace(alias_map)

# --- 3) Merge AUS side (City, Country) -> AUS_CityID ---
f5 = t5_long.merge(
    dim_citycountry.rename(columns={"City":"Australian_City", "Country":"Australian_Country", "CityID":"AUS_CityID"})[
        ["AUS_CityID","Australian_City","Australian_Country"]
    ],
    on=["Australian_City","Australian_Country"],
    how="left"
)

# --- 4) Merge foreign side (City, Country) -> FOR_CityID ---
f5 = f5.merge(
    dim_citycountry.rename(columns={"City":"Foreign_City", "Country":"Foreign_Country", "CityID":"FOR_CityID"})[
        ["FOR_CityID","Foreign_City","Foreign_Country"]
    ],
    on=["Foreign_City","Foreign_Country"],
    how="left"
)

# --- 5) Year-based DateID merge ---
dim_date["Year"] = pd.to_numeric(dim_date["Year"], errors="coerce").astype("Int64")
f5 = f5.merge(dim_date[["DateID","Year"]], on="Year", how="left")

# --- 6) Final fact (only keep rows where both city IDs exist) ---
fact_citypairs = (f5
    .dropna(subset=["AUS_CityID","FOR_CityID"])
    .rename(columns={"Value":"Amount"})[
        ["DateID","AUS_CityID","FOR_CityID","Metric","Direction","Amount"]
    ])

fact_citypairs.to_csv("../data/clean/fact_citypairs.csv", index=False)

# --- 7) Diagnostics ---
print("✅ fact_citypairs rows:", len(fact_citypairs))
print("⚠️ AUS_CityID missing (pre-drop):", f5["AUS_CityID"].isna().sum())
print("⚠️ FOR_CityID missing (pre-drop):", f5["FOR_CityID"].isna().sum())
print("⚠️ DateID missing:", fact_citypairs["DateID"].isna().sum())

# Show top unmatched AUS/Foreign city labels to build alias_map if needed
unmatched_aus = (f5[f5["AUS_CityID"].isna()]
                 .groupby("Australian_City").size()
                 .sort_values(ascending=False).head(20))
unmatched_for = (f5[f5["FOR_CityID"].isna()]
                 .groupby("Foreign_City").size()
                 .sort_values(ascending=False).head(20))
print("\nTop unmatched Australian_City values:\n", unmatched_aus.to_string())
print("\nTop unmatched Foreign_City values:\n", unmatched_for.to_string())

✅ fact_citypairs rows: 17664
⚠️ AUS_CityID missing (pre-drop): 0
⚠️ FOR_CityID missing (pre-drop): 0
⚠️ DateID missing: 0

Top unmatched Australian_City values:
 Series([], )

Top unmatched Foreign_City values:
 Series([], )


fact market share

Sanity checks

In [51]:
# --- Sanity checks for cleaned aviation model ---
from pathlib import Path
import pandas as pd
import numpy as np

BASE = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
CLEAN = BASE / "data" / "clean"

# ---------- Load ----------
def load_csv(path):
    df = pd.read_csv(path)
    print(f"Loaded {path.name:40s} rows={len(df)} cols={len(df.columns)}")
    return df

dim_airline        = load_csv(CLEAN / "dim_airline.csv")              # AirlineID, Airline_Name
dim_citycountry    = load_csv(CLEAN / "dim_citycountry.csv")          # CityID, City, Country
dim_date           = load_csv(CLEAN / "dim_date.csv")                 # DateID, Year, Month, YearMonthDate

fact_airline_monthly = load_csv(CLEAN / "fact_airline_monthly.csv")   # DateID, AirlineID, Country, Direction, Passengers, Freight_tonnes, Mail_tonnes
fact_airline_flights = load_csv(CLEAN / "fact_airline_flights.csv")   # DateID, AirlineID, Service_Region, Direction, Flights, Passengers, Seats_Available, Seat_Utilisation
fact_airport_totals  = load_csv(CLEAN / "fact_airport_totals.csv")    # DateID, CityID, Metric, Direction, Amount
fact_citypairs       = load_csv(CLEAN / "fact_citypairs.csv")         # DateID, AUS_CityID, FOR_CityID, Metric, Direction, Amount

# ---------- Helper reporting ----------
def header(t): print(f"\n{'='*12} {t} {'='*12}")
def show(df, n=5): 
    display(df.head(n).style.hide(axis='index'))

def check_columns(df, required, name):
    header(f"[{name}] required columns")
    missing = [c for c in required if c not in df.columns]
    extra   = [c for c in df.columns if c not in required]
    print("Missing:", missing)
    print("Extra  :", extra if extra else "None")

def check_unique(df, keys, name):
    header(f"[{name}] unique key {keys}")
    dups = df.duplicated(subset=keys, keep=False)
    print("Duplicate key rows:", dups.sum())
    if dups.any(): show(df.loc[dups].head(10))

def check_nulls(df, cols, name):
    header(f"[{name}] nulls in {cols}")
    nulls = df[cols].isna().sum().sort_values(ascending=False)
    display(nulls.to_frame("null_count"))

def check_fk(fact, fact_cols, dim, dim_key, name, dim_name):
    """fact_cols: list of columns in fact that must exist in dim[dim_key]"""
    header(f"[{name}] FK → {dim_name}")
    dim_keys = set(dim[dim_key].dropna().unique())
    for col in fact_cols:
        missing = ~fact[col].isin(dim_keys)
        cnt = missing.sum()
        print(f"{col}: missing keys = {cnt}")
        if cnt:
            show(fact.loc[missing, [col]].drop_duplicates().head(10))

def check_nonnegative(df, cols, name):
    header(f"[{name}] non-negative checks")
    for c in cols:
        if c in df:
            bad = (df[c] < 0) & df[c].notna()
            print(f"{c}: negatives = {bad.sum()}")
            if bad.any(): show(df.loc[bad, [c]].head(10))

# ---------- Expected schemas ----------
cols_dim_airline = ["AirlineID","Airline_Name"]
cols_dim_city    = ["CityID","City","Country"]
cols_dim_date    = ["DateID","Year","Month"]  # plus YearMonthDate allowed

cols_f1 = ["DateID","AirlineID","Country","Direction","Passengers","Freight_tonnes","Mail_tonnes"]
cols_f3 = ["DateID","AirlineID","Service_Region","Direction","Flights","Passengers","Seats_Available","Seat_Utilisation"]
cols_f4 = ["DateID","CityID","Metric","Direction","Amount"]
cols_f5 = ["DateID","AUS_CityID","FOR_CityID","Metric","Direction","Amount"]

# ---------- Column checks ----------
check_columns(dim_airline, cols_dim_airline, "dim_airline")
check_columns(dim_citycountry, cols_dim_city, "dim_citycountry")
check_columns(dim_date, cols_dim_date, "dim_date")

check_columns(fact_airline_monthly, cols_f1, "fact_airline_monthly")
check_columns(fact_airline_flights, cols_f3, "fact_airline_flights")
check_columns(fact_airport_totals, cols_f4, "fact_airport_totals")
check_columns(fact_citypairs, cols_f5, "fact_citypairs")

# ---------- Key uniqueness ----------
check_unique(dim_airline, ["AirlineID"], "dim_airline")
check_unique(dim_citycountry, ["CityID"], "dim_citycountry")
check_unique(dim_date, ["DateID"], "dim_date")

# ---------- Nulls on key columns ----------
check_nulls(dim_airline, ["AirlineID","Airline_Name"], "dim_airline")
check_nulls(dim_citycountry, ["CityID","City","Country"], "dim_citycountry")
check_nulls(dim_date, ["DateID","Year","Month"], "dim_date")

check_nulls(fact_airline_monthly, ["DateID","AirlineID","Direction","Passengers"], "fact_airline_monthly")
check_nulls(fact_airline_flights, ["DateID","AirlineID","Direction","Flights","Seats_Available"], "fact_airline_flights")
check_nulls(fact_airport_totals, ["DateID","CityID","Metric","Direction","Amount"], "fact_airport_totals")
check_nulls(fact_citypairs, ["DateID","AUS_CityID","FOR_CityID","Metric","Direction","Amount"], "fact_citypairs")

# ---------- FK integrity ----------
check_fk(fact_airline_monthly, ["DateID"], dim_date, "DateID", "fact_airline_monthly", "dim_date")
check_fk(fact_airline_monthly, ["AirlineID"], dim_airline, "AirlineID", "fact_airline_monthly", "dim_airline")

check_fk(fact_airline_flights, ["DateID"], dim_date, "DateID", "fact_airline_flights", "dim_date")
check_fk(fact_airline_flights, ["AirlineID"], dim_airline, "AirlineID", "fact_airline_flights", "dim_airline")

check_fk(fact_airport_totals, ["DateID"], dim_date, "DateID", "fact_airport_totals", "dim_date")
check_fk(fact_airport_totals, ["CityID"], dim_citycountry, "CityID", "fact_airport_totals", "dim_citycountry")

check_fk(fact_citypairs, ["DateID"], dim_date, "DateID", "fact_citypairs", "dim_date")
check_fk(fact_citypairs, ["AUS_CityID"], dim_citycountry, "CityID", "fact_citypairs", "dim_citycountry")
check_fk(fact_citypairs, ["FOR_CityID"], dim_citycountry, "CityID", "fact_citypairs", "dim_citycountry")

# ---------- Value/range checks ----------
check_nonnegative(fact_airline_monthly, ["Passengers","Freight_tonnes","Mail_tonnes"], "fact_airline_monthly")
check_nonnegative(fact_airline_flights, ["Flights","Passengers","Seats_Available"], "fact_airline_flights")
check_nonnegative(fact_airport_totals, ["Amount"], "fact_airport_totals")
check_nonnegative(fact_citypairs, ["Amount"], "fact_citypairs")

header("[fact_airline_flights] seat utilisation 0..1")
if "Seat_Utilisation" in fact_airline_flights:
    out_of_bounds = fact_airline_flights["Seat_Utilisation"].dropna().pipe(lambda s: (s<0) | (s>1)).sum()
    print("Seat_Utilisation outside [0,1]:", out_of_bounds)
    if out_of_bounds:
        show(fact_airline_flights.loc[(fact_airline_flights["Seat_Utilisation"]<0) | (fact_airline_flights["Seat_Utilisation"]>1)].head(10))

# ---------- Basic reconciliation snapshots ----------
header("Reconciliation snapshots")

# 1) Passengers by airline in May-2025 across tables 1 & 3 (if both have May-2025)
def label_for_dateid(did):
    row = dim_date.loc[dim_date["DateID"]==did]
    if row.empty: return "?"
    return f'{int(row["Year"].values[0])}-{int(row["Month"].values[0]):02d}'

# Pick the latest DateID available
latest_dateid = dim_date["DateID"].max() if "DateID" in dim_date else None
print("Latest DateID:", latest_dateid, label_for_dateid(latest_dateid) if latest_dateid is not None else "")

if latest_dateid is not None:
    f1_latest = fact_airline_monthly.query("DateID == @latest_dateid")
    f3_latest = fact_airline_flights.query("DateID == @latest_dateid")

    pax_by_airline_f1 = (f1_latest.groupby("AirlineID")["Passengers"].sum().rename("pax_f1"))
    pax_by_airline_f3 = (f3_latest.groupby("AirlineID")["Passengers"].sum().rename("pax_f3"))

    recon = (pax_by_airline_f1.to_frame()
             .merge(pax_by_airline_f3.to_frame(), left_index=True, right_index=True, how="outer")
             .fillna(0.0))
    recon["diff"] = recon["pax_f3"] - recon["pax_f1"]
    recon["abs_diff_pct"] = np.where(recon["pax_f1"]>0, abs(recon["diff"])/recon["pax_f1"], np.nan)
    print("Airline-level pax reconciliation (latest month) — top 10 abs diff:")
    show(recon.sort_values("abs_diff_pct", ascending=False).head(10))

# 2) City totals check: citypairs (sum of inbound/outbound passengers) vs airport totals (Passengers metric)
if "Metric" in fact_citypairs.columns and "Metric" in fact_airport_totals.columns:
    # passengers only
    cp_pax = fact_citypairs.query('Metric == "Passengers"')
    at_pax = fact_airport_totals.query('Metric == "Passengers"')

    # inbound to an AUS city = sum over foreign origins where Direction == 'Inbound'
    cp_in = cp_pax.query('Direction == "Inbound"').groupby(["DateID","AUS_CityID"])["Amount"].sum().rename("cp_in")
    at_in = at_pax.query('Direction == "Inbound"').groupby(["DateID","CityID"])["Amount"].sum().rename("at_in")

    # outbound from an AUS city
    cp_out = cp_pax.query('Direction == "Outbound"').groupby(["DateID","AUS_CityID"])["Amount"].sum().rename("cp_out")
    at_out = at_pax.query('Direction == "Outbound"').groupby(["DateID","CityID"])["Amount"].sum().rename("at_out")

    rec_in  = (cp_in.reset_index()
               .merge(at_in.reset_index(), left_on=["DateID","AUS_CityID"], right_on=["DateID","CityID"], how="inner"))
    rec_out = (cp_out.reset_index()
               .merge(at_out.reset_index(), left_on=["DateID","AUS_CityID"], right_on=["DateID","CityID"], how="inner"))

    for label, rec, left, right in [
        ("Inbound", rec_in,  "cp_in",  "at_in"),
        ("Outbound", rec_out,"cp_out", "at_out"),
    ]:
        header(f"City totals reconciliation — {label}")
        if not rec.empty:
            rec["diff"] = rec[left] - rec[right]
            rec["abs_diff_pct"] = np.where(rec[right]>0, abs(rec["diff"])/rec[right], np.nan)
            show(rec.sort_values("abs_diff_pct", ascending=False).head(10))
        else:
            print("No overlap to reconcile (check Metric/Direction labels).")

print("\n✅ Sanity checks complete.")

Loaded dim_airline.csv                          rows=97 cols=3
Loaded dim_citycountry.csv                      rows=84 cols=3
Loaded dim_date.csv                             rows=204 cols=6
Loaded fact_airline_monthly.csv                 rows=238 cols=7
Loaded fact_airline_flights.csv                 rows=222 cols=8
Loaded fact_airport_totals.csv                  rows=144 cols=6
Loaded fact_citypairs.csv                       rows=17664 cols=6

============ [dim_airline] required columns ============
Missing: []
Extra  : ['Home_Country']

============ [dim_citycountry] required columns ============
Missing: []
Extra  : None

============ [dim_date] required columns ============
Missing: []
Extra  : ['YearMonthDate', 'MonthName', 'Quarter']

============ [fact_airline_monthly] required columns ============
Missing: []
Extra  : None

============ [fact_airline_flights] required columns ============
Missing: []
Extra  : None

============ [fact_airport_totals] required columns ===========

,null_count
AirlineID,0
Airline_Name,0



============ [dim_citycountry] nulls in ['CityID', 'City', 'Country'] ============


,null_count
CityID,0
City,0
Country,0



============ [dim_date] nulls in ['DateID', 'Year', 'Month'] ============


...


============ City totals reconciliation — Inbound ============


DateID,AUS_CityID,cp_in,CityID,at_in,diff,abs_diff_pct
202401,1,35227.000000,1,35227.000000,0.000000,0.000000
202401,2,221626.000000,2,221626.000000,0.000000,0.000000
202501,10,1576.000000,10,1576.000000,0.000000,0.000000
202501,9,201195.000000,9,201195.000000,0.000000,0.000000
202501,8,449863.000000,8,449863.000000,0.000000,0.000000



============ City totals reconciliation — Outbound ============


DateID,AUS_CityID,cp_out,CityID,at_out,diff,abs_diff_pct
202401,1,37267.000000,1,37267.000000,0.000000,0.000000
202401,2,231012.000000,2,231012.000000,0.000000,0.000000
202501,10,1507.000000,10,1507.000000,0.000000,0.000000
202501,9,206834.000000,9,206834.000000,0.000000,0.000000
202501,8,460578.000000,8,460578.000000,0.000000,0.000000



✅ Sanity checks complete.
